In [1]:
def calculate_average_mass(df_row):
    sum_of_weights = 0

    for i in range(1,len(df_row)):
        sum_of_weights += float(df_row[i])
        average = sum_of_weights / (len(df_row) - 1)
    return "{:.4f}".format(average)

def rename_image(df_row):
    newname = df_row['Image'].replace("testname", "")

    return newname

def get_index(df_row):
    return df_row['Image'][:len(df_row['Image']) - 4]

In [19]:
weightsFileName = '/Users/pablo/Documents/Programacion/BlueberriesImageMassDataset/weightsBlueberries.txt'

import pandas as pd

df = pd.read_csv(weightsFileName, header=None, delim_whitespace=True)

df.rename(columns = {0:'Image'}, 
            inplace = True)

df['AverageMass'] = df.apply(calculate_average_mass, axis=1)

df['Image'] = df.apply(rename_image, axis=1)

df['ID'] = df.apply(get_index, axis=1)

mass = pd.DataFrame().assign(Image=df['Image'], ID=df['ID'], 
                 
                            AverageMass=df['AverageMass'])
mass['ID'] = mass['ID'].astype(int)
mass['AverageMass'] = mass['AverageMass'].astype(float)
#mass.to_json('massBlues.json', orient = 'split', compression = 'infer', index = 'true')
mass.head(10)
#latex_table = mass.head(10).to_latex(index=False, bold_rows=True, caption="Table of image Weighs and Average Mass")

#print(latex_table)



,Image,ID,AverageMass
0,21.jpg,21,10.2444
1,22.jpg,22,10.2427
2,23.jpg,23,10.2425
3,24.jpg,24,10.1629
4,25.jpg,25,10.1636
5,26.jpg,26,9.7124
6,27.jpg,27,9.7125
7,28.jpg,28,9.8781
8,29.jpg,29,9.8802
9,30.jpg,30,9.9806


In [36]:
IDAndMass = {}
for i in range(len(mass)):
    #print(mass.loc[i].at["ID"])
    IDAndMass[mass.loc[i].at["ID"]] = mass.loc[i].at["AverageMass"]

print(IDAndMass)

print(IDAndMass[21])


{21: 10.2444, 22: 10.2427, 23: 10.2425, 24: 10.1629, 25: 10.1636, 26: 9.7124, 27: 9.7125, 28: 9.8781, 29: 9.8802, 30: 9.9806, 31: 9.9795, 32: 9.1594, 33: 9.1617, 34: 10.8762, 35: 10.8735, 36: 10.8766, 37: 9.9834, 38: 9.9846, 39: 9.9304, 40: 9.9257, 41: 9.6898, 42: 9.6972, 43: 9.7016, 44: 11.9737, 45: 11.9794, 46: 8.7405, 47: 8.7423, 48: 9.1772, 49: 9.1813, 50: 9.4972, 51: 9.495, 52: 11.0753, 53: 11.0766, 54: 10.27, 55: 10.2695, 56: 11.5706, 57: 11.5745, 59: 8.7329, 60: 8.7356, 61: 10.2567, 62: 10.2568, 63: 10.213, 64: 10.2143, 65: 12.3145, 66: 12.3159, 67: 8.9281, 68: 8.9272, 70: 9.6905, 71: 9.6184, 72: 9.6184, 73: 10.6453, 74: 10.6412, 75: 8.8254, 76: 8.8253, 77: 9.9625, 78: 9.962, 79: 9.4596, 80: 9.4601, 81: 9.4612, 82: 10.532, 83: 10.5355, 84: 9.6529, 85: 9.6556, 86: 9.8557, 87: 9.8554, 88: 9.2088, 89: 9.2059, 90: 10.187, 91: 10.1867, 92: 10.0272, 93: 10.0269, 94: 9.3013, 95: 9.3014, 96: 11.002, 97: 9.5888, 98: 9.5873, 99: 9.7797, 100: 9.7792, 101: 9.6791, 102: 9.6792, 103: 10.6976,

In [33]:
import os
import glob
import cv2 #if you need to preprocess the images
import numpy as np
import re
from PIL import Image
from numpy import asarray

# Set the paths to your image and label directories
image_dir = "/Users/pablo/Documents/Programacion/BlueberriesImageMassDataset/Blueberries140/train/images"
regexpattern = r"testname(\d+)"

imagesdict = {}

for image_file in os.listdir(image_dir):
    #print(image_file)
    if image_file.endswith(".jpg"):
        image_path = os.path.join(image_dir, image_file)

        #image = cv2.imread(image_path)
        image = cv2.imread(image_path)

        imageID = re.search(regexpattern, image_file).group(0)[8:]
        imagesdict[imageID] = image

aux = {int(k):v for k,v in imagesdict.items()}
imagesdict = aux

#print(imagesdict)


idimages = list(imagesdict.keys())
idimages.sort()
sorted_dict = {i: imagesdict[i] for i in idimages}
imagesdict = sorted_dict
#print(imagesdict)

#imagesdf = pd.DataFrame.from_dict(images, orient='index')
#imagesdf.head(3)

In [42]:
labels = []
images = []

idmasskeys = list(IDAndMass.keys())

for k in idmasskeys:
    if(k in IDAndMass and k in imagesdict):
        #print(k)
        labels.append(IDAndMass[k])
        images.append(imagesdict[k])


images = np.array(images)
labels = np.array(labels)


In [43]:

train_size = int(0.8 * len(images))
train_images, val_images = images[:train_size], images[train_size:]
train_labels, val_labels = labels[:train_size], labels[train_size:]

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


train_images = train_images / 255.0

model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(512, 512, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(1)
])


model.compile(loss='mse', optimizer='adam')

model.fit(images, labels, batch_size=32, epochs=10, validation_split=0.2)




Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-01 03:02:03.577998: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-01 03:02:03.578507: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2023-05-01 03:02:04.536030: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-01 03:02:04.811803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 4135835.2500

2023-05-01 03:02:18.974895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 15s 3s/step - loss: 4135835.2500 - val_loss: 304705.3438
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 148838.2188 - val_loss: 27310.3242
Epoch 3/10
4/4 [==============================] - 6s 2s/step - loss: 14490.4160 - val_loss: 895.0084
Epoch 4/10
4/4 [==============================] - 10s 3s/step - loss: 2639.6228 - val_loss: 59.0020
Epoch 5/10
4/4 [==============================] - 7s 2s/step - loss: 495.8028 - val_loss: 178.1532
Epoch 6/10
4/4 [==============================] - 10s 3s/step - loss: 84.8290 - val_loss: 173.5659
Epoch 7/10
4/4 [==============================] - 8s 2s/step - loss: 188.2445 - val_loss: 27.2227
Epoch 8/10
4/4 [==============================] - 6s 2s/step - loss: 24.0729 - val_loss: 74.3879
Epoch 9/10
4/4 [==============================] - 10s 2s/step - loss: 66.2118 - val_loss: 27.3760
Epoch 10/10
4/4 [==============================] - 8s 2s/step - loss: 9.2923 - val_loss: 8.6391
